Import libraries and useful functions. Code reused from week 4 exercises.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from sklearn import metrics

sns.set_style("whitegrid")

def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

def compute_confusion_matrix(target, pred, normalize=None, n_classes=10):
    return metrics.confusion_matrix(
        target.detach().cpu().numpy(),
        pred.detach().cpu().numpy(),
        labels=np.arange(n_classes),
        normalize=normalize
    )

def show_image(img):
    img = img.detach().cpu()
    img = img / 2 + 0.5   # unnormalize
    with sns.axes_style("white"):
        plt.figure(figsize=(8, 8))
        plt.imshow(img.permute((1, 2, 0)).numpy())
        plt.axis('off')
        plt.show()

Load the CIFAR-10 dataset and split it into train, validation, calibration and test sets.

In [ ]:
# The output of torchvision datasets are PIL images in the range [0, 1]. 
# We transform them to PyTorch tensors and rescale them to be in the range [-1, 1].
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

batch_size = 64

# Load full training dataset (50,000 samples)
full_train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)

# Split into train (40k), validation (5k) and calibration (10k) sets
train_size = 35000  # 35,000
valid_size = 5000 # 5,000
calib_size = 10000 # 10,000

train_set, valid_set, calib_set = torch.utils.data.random_split(
    full_train_set, [train_size, valid_size, calib_size]
)

# Test set remains unchanged
test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)

# DataLoaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
calib_loader = DataLoader(calib_set, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

# Map from class index to class name.
classes = {index: name for name, index in full_train_set.class_to_idx.items()}

In [ ]:
print("Training data")
print("Number of points:", len(train_set))
x, y = next(iter(train_loader))
print("Batch dimension (B x C x H x W):", x.shape)
print(f"Number of distinct labels: {len(set(train_set.dataset.targets))} (unique labels: {set(train_set.dataset.targets)})")

print("\nValidation data")
print("Number of points:", len(valid_set))
x, y = next(iter(valid_loader))
print("Batch dimension (B x C x H x W):", x.shape)
print(f"Number of distinct labels: {len(set(valid_set.dataset.targets))} (unique labels: {set(valid_set.dataset.targets)})")

print("\nCalibration data")
print("Number of points:", len(calib_set))
x, y = next(iter(calib_loader))
print("Batch dimension (B x C x H x W):", x.shape)
print(f"Number of distinct labels: {len(set(calib_set.dataset.targets))} (unique labels: {set(calib_set.dataset.targets)})")

print("\nTest data")
print("Number of points:", len(test_set))
x, y = next(iter(test_loader))
print("Batch dimension (B x C x H x W):", x.shape)
print(f"Number of distinct labels: {len(set(test_set.targets))} (unique labels: {set(test_set.targets)})")

n_classes = len(set(test_set.targets))

In [ ]:
# Get random training images and show them.
images, labels = next(iter(train_loader))
show_image(torchvision.utils.make_grid(images))

Define model architecture and forward pass.

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.activation_fn = nn.ReLU

        self.net = nn.Sequential(
            # CNN
            nn.Conv2d(3, 32, 5),
            self.activation_fn(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            self.activation_fn(),
            nn.MaxPool2d(2),
            #nn.Dropout(0.2),

            nn.Flatten(),

            # FFNN
            nn.LazyLinear(128),
            self.activation_fn(),
            nn.Linear(128, 128),
            self.activation_fn(),
            nn.Linear(128, 128),
            self.activation_fn(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)
    


#model = Model(n_classes)
#device = torch.device('cpu')  
#model.to(device)
#print(model)

Initialize grid for grid search and the loss function.

In [ ]:
from itertools import product

# Define grid
param_grid = {
    "optimizer": ["Adam", "SGD"],
    "lr": [1e-1, 1e-3, 1e-5]
    }

# Create iterable list of param combinations
keys = param_grid.keys()
values = param_grid.values()

combinations = list(product(*values))
print(combinations)

for combo in combinations:
    print(combo[0])

# Define loss function
loss_fn = nn.CrossEntropyLoss()  

Perform grid search cross-validation.

In [ ]:
num_epochs = 8 ## train model for longer
accuracy_every_steps = 500

best_accuracy = -np.inf
best_param = []

train_accuracies = []

# Loop over parameter combinations
for combo in combinations:
    model = Model(n_classes)
    device = torch.device('cpu')  
    model.to(device)
    model.train()

    step = 0
    valid_accuracies = []

    # Retrieve learning rate
    lr = combo[1]
    
    # Retrive optimizer
    if combo[0] == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=lr) 
        
    # Train model
    for epoch in range(num_epochs):
        
        train_accuracies_batches = []
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass, compute gradients, perform one training step.

            # Compute outputs
            output = model(inputs)

            # Compute loss
            loss = loss_fn(output, targets)

            # Zero gradients
            optimizer.zero_grad()

            # Backprop
            loss.backward()

            # One step of gradient descent
            optimizer.step()
            
            # Increment step counter
            step += 1
            
            # Compute accuracy.
            predictions = output.max(1)[1]
            train_accuracies_batches.append(accuracy(targets, predictions))
            
            # Compute trian accuracy every  steps.
            if step % accuracy_every_steps == 0:
                train_accuracies.append(np.mean(train_accuracies_batches))
                print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")

    # Perfrom cross validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs, targets = inputs.to(device), targets.to(device)
                
            # Forward pass, compute gradients, perform one training step.

            # Compute outputs
            output = model(inputs)
                
            # Increment step counter
            step += 1
                
            # Compute accuracy.
            predictions = output.max(1)[1]
            valid_accuracies.append(accuracy(targets, predictions))

        mean_valid_accuracy = np.mean(valid_accuracies)
        print(f"Validation accuracy for combo {combo}: {mean_valid_accuracy}")

        if mean_valid_accuracy > best_accuracy:
            best_accuracy = mean_valid_accuracy
            best_param = combo


print("Finished training.")
print("Best params:", best_param)
print("Best accuract:", best_accuracy)

Train the model using best parameters.

In [ ]:
lr = best_param[1]
if best_param[0] == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.CrossEntropyLoss()  

In [ ]:
num_epochs = 8 ## train model for longer
accuracy_every_steps = 500

step = 0

model = Model(n_classes)
device = torch.device('cpu')  
model.to(device)
model.train()

train_accuracies = []
valid_accuracies = []
        
for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass, compute gradients, perform one training step.

        # Compute outputs
        output = model(inputs)

        # Compute loss
        loss = loss_fn(output, targets)

        # Zero gradients
        optimizer.zero_grad()

        # Backprop
        loss.backward()

        # One step of gradient descent
        optimizer.step()
        
        # Increment step counter
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy(targets, predictions))
        
        # Validate every `validation_every_steps` steps.
        if step % accuracy_every_steps == 0:
            train_accuracies.append(np.mean(train_accuracies_batches))
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")


print("Finished training.")

In [ ]:
def conformal_prediction(calib_loader, model):
    true_label_prob = []
    alpha = 0.1

    with torch.no_grad():
        for inputs, targets in calib_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)

            # Compute softmax probabilities
            probs = torch.softmax(output, dim=1)

            # Add softmax probabilities for true labels into a list
            for sample in range(len(probs)):
                true_label_prob.append(probs[sample][targets[sample].item()])

        # Compute non-conformity scores
        non_comf_scores = np.zeros(len(true_label_prob))
        for i in range(len(true_label_prob)):
            non_comf_scores[i] = 1 - true_label_prob[i]
        non_comf_scores.sort()
            
        # Compute threshold
        threshold_index = np.ceil((len(non_comf_scores) + 1) * (1 - alpha)) - 1
        threshold = non_comf_scores[int(threshold_index)]
        print(threshold)

    return threshold
    
thresh = conformal_prediction(calib_loader, model)

Evaluate model performance on test set.

In [ ]:
# Evaluate test set
confusion_matrix = np.zeros((n_classes, n_classes))
with torch.no_grad():
    model.eval()
    test_accuracies = []
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        output = model(inputs)

        probs = torch.softmax(output, dim=1)

        # Prediction set per batch
        pred_set = []

        # Add classes to prediction set
        for i in range(len(probs)):
            for j in range(len(probs[i])):
                if probs[i][j] >= 1 - thresh:
                    pred_set.append(j)

            print("possible classes", pred_set)
            print("true class", targets[i].item())
            pred_set = []

            ## ADD OTHER DIAGNOSTICS FOR PROPER EVALUATION OF THE CONFORMAL PREDICTIONS
        

        loss = loss_fn(output, targets)

        predictions = output.max(1)[1]

        # Multiply by len(inputs) because the final batch of DataLoader may be smaller (drop_last=True).
        test_accuracies.append(accuracy(targets, predictions) * len(inputs))
        
        confusion_matrix += compute_confusion_matrix(targets, predictions)

    test_accuracy = np.sum(test_accuracies) / len(test_set)
    
    model.train()

print(f"Test accuracy: {test_accuracy:.3f}")

UPDATE PLOTS TO INCLUDE CONFORMAL PREDICTION

Plot confusion matrix for test set.

In [ ]:
def normalize(matrix, axis):
    axis = {'true': 1, 'pred': 0}[axis]
    return matrix / matrix.sum(axis=axis, keepdims=True)

x_labels = [classes[i] for i in classes]
y_labels = x_labels
plt.figure(figsize=(6, 6))
sns.heatmap(
    ax=plt.gca(),
    data=normalize(confusion_matrix, 'true'),
    annot=True,
    linewidths=0.5,
    cmap="Reds",
    cbar=False,
    fmt=".2f",
    xticklabels=x_labels,
    yticklabels=y_labels,
)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.ylabel("True class")
plt.xlabel("Predicted class")
plt.tight_layout()
plt.show()

Plot per class accuracy of predictions.

In [ ]:
with sns.axes_style('whitegrid'):
    plt.figure(figsize=(8, 4))
    sns.barplot(x=x_labels, y=np.diag(normalize(confusion_matrix, 'true')))
    plt.xticks(rotation=90)
    plt.title("Per-class accuracy")
    plt.ylabel("Accuracy")
    plt.show()